In [ ]:
import cv2
import os
import pydicom as dicom
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
path='./temp_TCIA'

In [ ]:
'''
###change data format(DICOM->PNG or JPG)###
PNG=True
for pat_dir in os.listdir(path):
    for x in os.listdir(os.path.join(path,pat_dir,'CT')):
        #load .dcm file
        dcm_img=dicom.dcmread(os.path.join(path,pat_dir,'CT',x))
        #scale pixel
        pix_arr=dcm_img.pixel_array.astype(float)
        pix_arr_scaled=(np.maximum(pix_arr,0) / pix_arr.max()) * 255.0
        pix_arr_scaled=np.uint8(pix_arr_scaled)
        #change format: .dcm to .png or .jpg
        if PNG == False:
            x = x.replace('.dcm', '.jpg')
        else:
            x = x.replace('.dcm', '.png')
        cv2.imwrite(os.path.join(path,pat_dir,'CT',x),pix_arr_scaled)
'''

In [ ]:
dict_dcm_IN = {}
tmp_sorted_IN = []
for pat_dir in os.listdir(path):
    for x in os.listdir(os.path.join(path, pat_dir, 'CT')):
        #loading DCM file's Instance Number
        fileName, fileExt = os.path.splitext(x) #splilt file into file's name and file's esxtension name
        if fileExt == '.dcm':
            dcm_file = dicom.dcmread(os.path.join(path, pat_dir, 'CT', x))
            dcm_IN = int(dcm_file.InstanceNumber) #instance num, convert to int for sorting
            dict_dcm_IN.update({fileName : dcm_IN}) #file'name : instance num
            tmp_sorted_IN = sorted(dict_dcm_IN.items(), key = lambda item:item[1]) #sorted with instance num(value), file's name : instance num
        else:
            continue
    for i in range(len(tmp_sorted_IN)):
        dcm_img = dicom.read_file(os.path.join(path, pat_dir, 'CT', tmp_sorted_IN[i][0]+'.dcm'))
        pix_arr = dcm_img.pixel_array.astype(float)
        pix_arr_scaled = (np.maximum(pix_arr, 0) / pix_arr.max()) * 255.0
        pix_arr_scaled = np.uint8(pix_arr_scaled)
        cv2.imwrite(os.path.join(path, pat_dir, 'CT', 'CT_' + str(i+1).zfill(3) + '.png'), pix_arr_scaled) #rename connected with mask file's name
    dict_dcm_IN = {} #reset to reuse
    tmp_sorted_IN = [] #reset to reuse
#print(dict_dcm_IN)
#print("----------")
#print(tmp_sorted_IN)
#print("done")

In [ ]:
#bestImg={patient dir:mask filname}
#bestSize={patient dir:max size of nodule}
bestImg={}
bestSize={}
maxDia={} #maxDia={mask filename:size of nodule}
for pat_dir in os.listdir(path):
    for x in os.listdir(os.path.join(path,pat_dir,'MASK')):
        _,ext=os.path.splitext(x)
        if ext=='.png' or ext=='.jpg':
            img=cv2.imread(os.path.join(path,pat_dir,'MASK',x),cv2.IMREAD_GRAYSCALE)
        else:
            continue
        '''
        numWhitePix = 0 
        for i in range(img.shape[0]):
            for j in range(img.shape[1]):
                if img[i][j]==255:
                    numWhitePix+=1
        '''
        mask_contours,_=cv2.findContours(img, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        if not mask_contours:
            continue
        contoursX=[]
        contoursY=[]
        for i in range(len(mask_contours)):
            mask_contours[i].shape=(len(mask_contours[i]),2)
            for j in range(len(mask_contours[i])):
                contoursX.append(mask_contours[i][j][0])
                contoursY.append(mask_contours[i][j][1])
        #print(contoursX)
        max_dia=(max(contoursX)-min(contoursX)) if (max(contoursX)-min(contoursX))>(max(contoursY)-min(contoursY)) else (max(contoursY)-min(contoursY))
        maxDia.update({x : max_dia})
    #print(pat_dir,': ',max(maxDia,key=maxDia.get),max(maxDia.values()))
    bestImg.update({pat_dir:max(maxDia,key=maxDia.get)})
    bestSize.update({pat_dir:max(maxDia.values())})
    #print(pat_dir,": ",maxDia)
    maxDia={}
patch_size=max(bestSize.values())
#print(bestImg)
#print(bestSize)
#print(patch_size)

In [ ]:
'''
###decide patch size###
#find the patient who have biggest nodule and the mask filename of that patient
max_img_patient=max(bestSize,key=bestSize.get)
max_img_filename=bestImg[max_img_patient]
#load mask image
max_img=cv2.imread(os.path.join(path,max_img_patient,'MASK',max_img_filename),cv2.IMREAD_GRAYSCALE)
#find contours of that image
contours,hierarchy= cv2.findContours(max_img, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
#append all x and y value of contour to list

patch_size=0
x=[]
y=[]
for i in range(len(contours)):
    contours[i].shape=(len(contours[i]),2)
    for j in range(len(contours[i])):
        x.append(contours[i][j][0])
        y.append(contours[i][j][1])
#print((max(x)-min(x)),(max(y)-min(y)))
#patch size = max(x)-min(x)+40 or max(y)-min(y)+40
patch_size=(max(x)-min(x))+50 if (max(x)-min(x))>(max(y)-min(y)) else (max(y)-min(y))+50
print('patch_size: ',patch_size)
'''

In [ ]:
#create a patch directory per every patient
for pat_dir in list(os.listdir(path)):
    try:
        if not(os.path.isdir(os.path.join(path,pat_dir,'patch'))):
            os.makedirs(os.path.join(path,pat_dir,'patch'))
    except OSError as e:
        if e.errno != errno.EEXIST:
            print("Failed to create directory!!!!!")
            raise

In [ ]:
###create a patch file per every patient###
for pat_dir in list(os.listdir(path)):
    #mask filename, ct filename, patch filename
    mask_filename,ext=os.path.splitext(bestImg[pat_dir])
    ct_filename=mask_filename.replace('mask','CT')+'.png'
    patch_filename=mask_filename.replace('mask','patch')+'.png'
    
    #load mask and ct files
    mask=cv2.imread(os.path.join(path,pat_dir,'MASK',bestImg[pat_dir]),cv2.IMREAD_GRAYSCALE)
    ct=cv2.imread(os.path.join(path,pat_dir,'CT',ct_filename),cv2.IMREAD_GRAYSCALE)  

    #find center coordinates of nodule 
    M=cv2.moments(mask)
    cX = int(M["m10"] / M["m00"])
    cY = int(M["m01"] / M["m00"])
    
    #crop ct image depending on its center coordinates(=create the patch)

    patch=ct[cY-int(patch_size/2):cY+int(patch_size/2),cX-int(patch_size/2):cX+int(patch_size/2)]

    patch_mask=mask[cY-int(patch_size/2):cY+int(patch_size/2),cX-int(patch_size/2):cX+int(patch_size/2)]
    
    #write the patch file on the patch directory
    cv2.imwrite(os.path.join(path,pat_dir,'patch',patch_filename),patch)
    #plt.imshow(patch,cmap='gray')
    #plt.imshow(ct,cmap='gray')
    #print(pat_dir,mask_filename)
    #lt.imshow(patch_mask,cmap='gray')